# Generate Training Data

We wish to speed up the process of having a human being name entities that they recognize in a paragraph, and having their positions in the paragraph identified and placed in a syntax usable by spaCy's NER training routine.

## Open PDF file, extract page two and display as sentences

In [6]:
import spacy
import PyPDF2
nlp = spacy.load('en_core_web_sm')

#Open PDF file for reading
pdfFile = open("BarCvDescLJ11.pdf", mode="rb")
pdfReader = PyPDF2.PdfFileReader(pdfFile)

# Select a page to work on
pageNumber = 12

# Get text
OnePage = pdfReader.getPage(pageNumber-1) #0-based count
OnePageText = OnePage.extractText()

# Close PDF file
pdfFile.close()

# Remove newlinesxmx82k-&a. It appears multiple newlines together makes
# Spacy think that is the end of a sentence. The PDF reader reads the text in
# an odd fashion
OnePageText = OnePageText.replace('\n','')

# create a spaCy doc object from the page and break it into sentences
doc = nlp(OnePageText)
l=0
for sent in doc.sents:
     print(l, ": ", sent)
     l = l+1


0 :  12  intermountain region of northern California.     
1 :  BANCROFT  
2 :  Bancroft is a two-rowed spring feed barley.
3 :  It was developed cooperatively by the USDA-ARS and the Idaho, Colorado, and Oregon AESs and released in 2000.
4 :  It was selected from the cross Hector/60Ab1810-53.
5 :  Its experimental designation was 78Ab10274.
6 :  It is midseason in maturity, 3 days earlier than Crystal and 2 days earlier than Harrington.
7 :  It is mid-tall, similar in plant height to Crystal and Harrington, but inferior in straw strength.  
8 :  Spikes are lax.
9 :  Awns are rough.
10 :  Kernels are covered and have white aleurone.
11 :  Rachilla hairs are long.
12 :  Barbs on lateral veins are few to none.
13 :  Lemmas are typically wrinkled.
14 :  Glumes are covered with long hairs.
15 :  Rachis edges have numerous hairs.
16 :  At the time of release it was resistant to stripe rust.
17 :  It was evaluated as Entry 1014 in the UC Regional Cereal Testing program in 1998 for spring pla

## Read in Per-line named entity file and match entities to sentence positions.

In [8]:
import re
import csv
import pandas as pd
fname = "Data/barley_p12_ner.txt"

# Covert the nlp senetence generator into a list of sentences
sentences = list(doc.sents)

# Open the file of manually matched pairs (sentence # <tab> word phrase <tab> named entity)
# e.g.:
#  0      AC Metcalfe     CVAR
#  0      two-rowed       TRAT
#  0      barley          CROP
#  1      Agri-Food Candada   ORG
#  1      1997    DATE
file = open(fname)
reader = csv.reader(file, delimiter='\t', quoting=csv.QUOTE_NONE)
data = list()

for row in reader:
    try:
        (sentIndex, phrase, label) = row
        sent = sentences[int(sentIndex)].string.rstrip()
        
        # find all instances of the 'phrase' in the 'sent'.
        iter = re.finditer(r"\b"+phrase+r"\b", sent)
        indices = [m.start(0) for m in iter]
        
        # check to make sure the phrase the user said was there was indeed found
        if len(indices) == 0:
            raise ValueError
                
        # print out all instances
        for i in indices:
#            print(sentIndex, sent, phrase, "("+str(i), i+len(phrase), "'"+label+"')")
            data.append([sentIndex, sent, phrase, "("+str(i)+", "+str(i+len(phrase))+", '"+label+"')"])
            
    except:
        print("Handle manually: ", row)
        
df = pd.DataFrame(data, columns = ["Index", "Sentence", "Phrase", "MatchInfo"])
print(df)


Handle manually:  ['18', 'Crop Science 41:265-266 (2001)', 'JRNL']
Handle manually:  ['40', 'Crop Science 30(2): 421 (1990)', 'JRNL']
    Index                                           Sentence          Phrase  \
0       2        Bancroft is a two-rowed spring feed barley.        Bancroft   
1       2        Bancroft is a two-rowed spring feed barley.       two-rowed   
2       2        Bancroft is a two-rowed spring feed barley.          spring   
3       2        Bancroft is a two-rowed spring feed barley.            feed   
4       2        Bancroft is a two-rowed spring feed barley.          barley   
..    ...                                                ...             ...   
147    65  It has late maturity, short plant height, and ...  straw strength   
148    66  Spikes are long, initially erect and semi-droo...          Spikes   
149    66  Spikes are long, initially erect and semi-droo...     anthocyanin   
150    66  Spikes are long, initially erect and semi-droo...      

## Create a function to clean up overlapping intervals

In [9]:
import re
coordRegex = re.compile(r'(\d+), (\d+)')

def sortByStart(coords):
    """For use in sort routines, return object with lowest (X,Y) values"""
    # split out coordinates that come in as (5, 7, 'CVAR')
    mo = coordRegex.search(coords)
    return(int(mo.group(1)))

def overlaps(coord1, coord2):
    """Check if coordinates of the form 5, 7, 'CVAR' and 32, 46, 'TRAT' overlap"""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
    
    if ((coord1High >= coord2Low) and (coord1Low <= coord2Low) or
        (coord2High >= coord1Low) and (coord2Low <= coord1Low)):
        return True
    else:
        return False

def keepFirst(coord1, coord2):
    """Given overlapping coordinates, return the wider encompassing one."""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
 
    if (int(coord1High) - int(coord1Low)) >= (int(coord2High) - int(coord2Low)):
        return True
    else:
        return False

# print("Should be false:", overlaps("(5, 7, 'CVAR')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(26, 46, 'TRAT')"))
# print("Keeper:", keepFirst("34, 46, 'TRAT'", "34, 46, 'TRAT'"))

def cleanIntervals(inputString=""):
    """order intervals like (5, 7, 'CVAR'), (32, 46, 'TRAT'), (26, 46, 'TRAT') and remove overlapping ones."""
    inputString = inputString.lstrip("(").rstrip(")")
    intervalList = inputString.split("), (")
    intervalList.sort(key = sortByStart)
#    print("Sorted Interval List:", intervalList)

    # Pairwise compare every interval in the list to every other interval to check overlap
    keeperList = [True]*len(intervalList) # Logic array to determine if each interval should be kept
    i=0
    for interval1 in intervalList:
        for interval2 in intervalList:
            if interval1 == interval2:
                if intervalList.index(interval1) != i: # when both are the same we reject the higher one
                    keeperList[i] = False
            else:
                if overlaps(interval1, interval2) and keepFirst(interval1, interval2) == False:
                    keeperList[i] = False
        i = i+1
        
#    print("keeperList:", keeperList)
   
    # Build up the return interval list
    returnStr = "("
    for interval, isKeeper in zip(intervalList, keeperList):
        if isKeeper:
            returnStr = returnStr + interval + "), ("
    return (returnStr.rstrip("), (") + ")")
        
# cleanIntervals("(5, 7, 'CVAR'), (32, 46, 'TRAT'), (5, 9, 'CVAR'), (48, 55, 'ORG'), (26, 46, 'TRAT')")
# cleanIntervals("(0, 8, 'CVAR'), (0, 5, 'CVAR'), (21, 26, 'PLAN'), (32, 37, 'CVAR')")
cleanIntervals("(0, 12, 'CVAR'), (39, 49, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (140, 150, 'CVAR'), (181, 187, 'CVAR'), (181, 187, 'CVAR')")


"(0, 12, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (181, 187, 'CVAR')"

## Aggregate all matches for each sentence on a single line and output in spaCy training format

In [10]:
# use Pandas dataframes to aggregate all entity matches together for a single sentence
agg_rules = {'Sentence': 'first', 'Phrase': 'first', 'MatchInfo': lambda x: ', '.join(x)}
res = df.groupby('Index').agg(agg_rules)
#print(res)

# Now format it just like what is needed for the spaCy training module: 
# E.g.:
# ('Eight-Twelve is a six-rowed winter feed barley', {'entities': [(0, 12, 'CVAR'), (18, 27, 'TRAT'), (28, 39, 'TRAT'),(40, 46, 'CROP')]}),
records = res.to_dict('records')
for i in range(0,len(records)):
    print("    ('"+records[i]['Sentence']+"', {'entities': ["+cleanIntervals(records[i]['MatchInfo'])+"]}),")
    #print("    ('"+records[i]['Sentence']+"', {'entities': ["+records[i]['MatchInfo']+"]}),")


    ('Kernels are covered and have white aleurone.', {'entities': [(0, 7, 'PLAN'), (35, 43, 'PLAN')]}),
    ('Rachilla hairs are long.', {'entities': [(0, 14, 'PLAN')]}),
    ('Barbs on lateral veins are few to none.', {'entities': [(0, 5, 'PLAN'), (9, 22, 'PLAN')]}),
    ('Lemmas are typically wrinkled.', {'entities': [(0, 6, 'PLAN')]}),
    ('Glumes are covered with long hairs.', {'entities': [(0, 6, 'PLAN'), (29, 34, 'PLAN')]}),
    ('Rachis edges have numerous hairs.', {'entities': [(0, 12, 'PLAN'), (27, 32, 'PLAN')]}),
    ('At the time of release it was resistant to stripe rust.', {'entities': [(30, 39, 'PPTD'), (43, 54, 'PATH')]}),
    ('Bancroft is a two-rowed spring feed barley.', {'entities': [(0, 8, 'CVAR'), (14, 23, 'TRAT'), (24, 30, 'TRAT'), (31, 35, 'TRAT'), (36, 42, 'CROP')]}),
    ('Baretta is a six-rowed spring feed barley.', {'entities': [(0, 7, 'CVAR'), (13, 22, 'TRAT'), (23, 29, 'TRAT'), (30, 34, 'TRAT'), (35, 41, 'CROP')]}),
    ('It was released by Arizona Plant B